In [1]:
import os
import pandas as pd
preprocessing_folder_path = '/home/local/PSYCH-ADS/xuqian_chen/Github/twitter_entropy/code/preprocessing/'
preprocessed_folder_path = '/home/local/PSYCH-ADS/xuqian_chen/Github/twitter_entropy/data/preprocessed/'
usdf = pd.read_csv(os.path.join(preprocessed_folder_path, 'us.csv'))
jpdf = pd.read_csv(os.path.join(preprocessed_folder_path, 'jp.csv'))


/tmp/ipykernel_1229626/3734358892.py:5: DtypeWarning: Columns (19,20,21,58) have mixed types. Specify dtype option on import or set low_memory=False.
  usdf = pd.read_csv(os.path.join(preprocessed_folder_path, 'us.csv'))
/tmp/ipykernel_1229626/3734358892.py:6: DtypeWarning: Columns (19,20,21,23,31,33,34,35,41) have mixed types. Specify dtype option on import or set low_memory=False.
  jpdf = pd.read_csv(os.path.join(preprocessed_folder_path, 'jp.csv'))


# entropy calculation

## load dict

In [2]:
def load_en_nrc_emotion_lexicon(txt_path):
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        
    data = [line.strip().split('\t') for line in lines]
    df = pd.DataFrame(data, columns=["word", "emotion", "value"])
    df["value"] = df["value"].astype(int)
    
    df = df[df["value"] == 1]
    emotion_dict = {emotion: set(df[df["emotion"] == emotion]["word"]) for emotion in df["emotion"].unique()}
    
    return emotion_dict

# path_to_english_nrc = os.path.join(preprocessing_folder_path, 'NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')
path_to_english_nrc = '/home/local/PSYCH-ADS/xuqian_chen/Github/twitter_entropy/data/preprocessing/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'
english_nrc_emotion_dict = load_en_nrc_emotion_lexicon(path_to_english_nrc)


def load_jp_nrc_emotion_lexicon(file_path):
    emotion_dict = {"anger": set(), "anticipation": set(), "disgust": set(), "fear": set(), "joy": set(),
                    "negative": set(), "positive": set(), "sadness": set(), "surprise": set(), "trust": set()}

    with open(file_path, encoding="utf-8") as f:
        next(f)  # Skip the header line
        for line in f:
            split_line = line.strip().split("\t")
            japanese_word = split_line[-1]
            emotions = [emotion for emotion, value in zip(emotion_dict.keys(), split_line[1:-1]) if int(value) == 1]

            for emotion in emotions:
                emotion_dict[emotion].add(japanese_word.lower())

    return emotion_dict

# path_to_japanese_nrc = os.path.join(preprocessing_folder_path, 'Japanese-NRC-EmoLex.txt')
path_to_japanese_nrc = '/home/local/PSYCH-ADS/xuqian_chen/Github/twitter_entropy/data/preprocessing/Japanese-NRC-EmoLex.txt'
japanese_nrc_emotion_dict = load_jp_nrc_emotion_lexicon(path_to_japanese_nrc)


# NOT IN USE: roberta senetiment analysis

In [3]:
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

def get_emotion_loadings(tweet_text):
    model_name = "cardiffnlp/twitter-roberta-base-emotion"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    emotion_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
    result = emotion_classifier(tweet_text)

    # Extract emotion probabilities
    emotion_probabilities = [entry["score"] for entry in result]
    return emotion_probabilities

# Example usage
tweet_text = "I love this new phone! It's amazing."
emotion_loadings = get_emotion_loadings(tweet_text)
print("Emotion loadings:", emotion_loadings)

def emotion_entropy(post_emotions, threshold=0.5):
    binary_emotions = [1 if emotion >= threshold else 0 for emotion in post_emotions]
    probabilities = [emotion_count / len(post_emotions) for emotion_count in binary_emotions]
    
    entropy = -np.sum([p * np.log2(p) if p > 0 else 0 for p in probabilities])
    return entropy

# Example usage
post_emotions = [0.7, 0.2, 0.8, 0.4, 0.6, 0.1]
entropy = emotion_entropy(post_emotions)
print("Emotion entropy:", entropy)

tweet_text = "I love this new phone! It's amazing."
emotion_loadings = get_emotion_loadings(tweet_text)
entropy = emotion_entropy(emotion_loadings)
print("Emotion loadings:", emotion_loadings)
print("Emotion entropy:", entropy)


2023-06-20 22:26:10.963327: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-20 22:26:10.963347: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Emotion loadings: [0.9440009593963623]
Emotion entropy: 1.292481250360578
Emotion loadings: [0.9440009593963623]
Emotion entropy: -0.0


# DDR

In [3]:
basic_emotions = ['anger', 'fear', 'joy', 'sadness', 'disgust', 'surprise']

ja_basic_emotion_concepts = {
    emotion: list(words) for emotion, words in japanese_nrc_emotion_dict.items() if emotion in basic_emotions
}
en_basic_emotion_concepts = {
    emotion: list(words) for emotion, words in english_nrc_emotion_dict.items() if emotion in basic_emotions
}

import fasttext
import fasttext.util
# check if the fastText embeddings are downloaded
import os

if not os.path.isfile('cc.ja.300.bin'):
    fasttext.util.download_model('ja', if_exists='ignore') 
if not os.path.isfile('cc.en.300.bin'):
    fasttext.util.download_model('en', if_exists='ignore')

ja_embeddings = fasttext.load_model('cc.ja.300.bin')
en_embeddings = fasttext.load_model('cc.en.300.bin')


## function: english emotions

In [8]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
import gensim.downloader as api

# load pre-trained fastText embeddings
def load_fasttext_embeddings(model_name):
    model = api.load(model_name)
    # Get the mean vector for the list of words
    # mean_vector = np.mean([model1[word] for word in wordlist if word in word_vectors], axis=0)

    return model

# Load pre-trained GloVe embeddings
def load_glove_embeddings(embeddings_file):
    embeddings = {}
    with open(embeddings_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)
            embeddings[word] = vector
    return embeddings

# Vectorize the dictionary of moral and emotional concepts
def vectorize_concepts(concepts, embeddings):
    concept_vectors = {}
    for concept, words in concepts.items():
        vectors = [embeddings[word] for word in words if word in embeddings]
        concept_vectors[concept] = np.mean(vectors, axis=0)
    return concept_vectors

# Vectorize the input text
def vectorize_text(text, embeddings):
    tokens = word_tokenize(text)
    vectors = [embeddings[token] for token in tokens if token in embeddings]
    return np.mean(vectors, axis=0)

# Calculate cosine similarity between input text and moral and emotional concepts
def calculate_similarities(text_vector, concept_vectors):
    similarities = {}
    for concept, vector in concept_vectors.items():
        similarities[concept] = cosine_similarity(text_vector.reshape(1, -1), vector.reshape(1, -1))[0][0]
    return similarities

# Example usage:
# glove_embeddings_file = 'glove.6B.300d.txt'
# glove_embeddings = load_glove_embeddings(glove_embeddings_file)
# en_embeddings = load_fasttext_embeddings("fasttext-wiki-news-subwords-300")

# Define your moral and emotional concepts dictionary here
concept_vectors = vectorize_concepts(en_basic_emotion_concepts, en_embeddings)

text = "This is an example text to analyze moral and emotional content."
text_vector = vectorize_text(text, en_embeddings)

similarities = calculate_similarities(text_vector, concept_vectors)
print(similarities)


{'fear': 0.41364282, 'sadness': 0.42352962, 'anger': 0.42670834, 'surprise': 0.48255157, 'disgust': 0.39483708, 'joy': 0.44914463}


# jp emotions

## NOT IN USE: Tokenize the input text using Janome

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
from janome.tokenizer import Tokenizer

# Load pre-trained FastText embeddings
def load_fasttext_embeddings(embeddings_file):
    embeddings = {}
    with open(embeddings_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)
            embeddings[word] = vector
    return embeddings

# Vectorize the dictionary of moral and emotional concepts
def vectorize_concepts(concepts, embeddings):
    concept_vectors = {}
    for concept, words in concepts.items():
        vectors = [embeddings[word] for word in words if word in embeddings]
        concept_vectors[concept] = np.mean(vectors, axis=0)
    return concept_vectors

# Tokenize the input text using Janome
def tokenize_japanese_text(text):
    tokenizer = Tokenizer()
    tokens = tokenizer.tokenize(text, wakati=True)
    return tokens

# Vectorize the input text
def vectorize_text(tokens, embeddings):
    vectors = [embeddings[token] for token in tokens if token in embeddings]
    return np.mean(vectors, axis=0)

# Calculate cosine similarity between input text and moral and emotional concepts
def calculate_similarities(text_vector, concept_vectors):
    similarities = {}
    for concept, vector in concept_vectors.items():
        similarities[concept] = cosine_similarity(text_vector.reshape(1, -1), vector.reshape(1, -1))[0][0]
    return similarities

# ENTROPY
def emotion_entropy(post_emotions, threshold=0.5):
    binary_emotions = [1 if emotion >= threshold else 0 for emotion in post_emotions]
    probabilities = [emotion_count / len(post_emotions) for emotion_count in binary_emotions]
    
    entropy = -np.sum([p * np.log2(p) if p > 0 else 0 for p in probabilities])
    return entropy
# Define your moral and emotional concepts dictionary here
concepts = en_basic_emotion_concepts

concept_vectors = vectorize_concepts(concepts, en_embeddings)

text = "His success is wonderful, but at the same time, I feel a bit jealous."
text_vector = vectorize_text(text, en_embeddings)

similarities = calculate_similarities(text_vector, concept_vectors)

entropy = emotion_entropy(list(similarities.values()))
print(similarities)
print(entropy)

{'fear': 0.28294072, 'sadness': 0.28086483, 'anger': 0.28085762, 'surprise': 0.30233887, 'disgust': 0.28768104, 'joy': 0.27653188}
0


In [10]:
import MeCab
def tokenize_japanese_text_mecab(text):
    mecab = MeCab.Tagger("-Owakati")
    return mecab.parse(text).strip().split()

def vectorize_jp_concepts(concepts, embeddings):
    concept_vectors = {}
    for concept, words in concepts.items():
        vectors = [ja_embeddings.get_word_vector(word) for word in words]
        concept_vectors[concept] = np.mean(vectors, axis=0)
    return concept_vectors

# Vectorize the input text
def vectorize_jp_text(tokens, embeddings):
    vectors =[ja_embeddings.get_word_vector(token) for token in tokens]
    return np.mean(vectors, axis=0)
# Example usage:
# fasttext_embeddings_file = 'cc.ja.300.vec'
# fasttext_embeddings = load_fasttext_embeddings(fasttext_embeddings_file)

# Define your moral and emotional concepts dictionary here
concept_vectors = vectorize_jp_concepts(ja_basic_emotion_concepts, ja_embeddings)

# concept_vectors = vectorize_concepts(concepts, ja_embeddings)

text = "これは道徳的および感情的なコンテンツを分析するための例文です"
tokens = tokenize_japanese_text_mecab(text)
text_vector = vectorize_jp_text(tokens, ja_embeddings)

similarities = calculate_similarities(text_vector, concept_vectors)

entropy = emotion_entropy(list(similarities.values()))
print(similarities)
print(entropy)

{'anger': 0.6377529, 'disgust': 0.611441, 'fear': 0.62469923, 'joy': 0.6755974, 'sadness': 0.6264248, 'surprise': 0.67802525}
2.584962500721156


In [11]:
en_mixed_emotion_sentences = [
    "I'm so happy to see you, but I'm sad that you have to leave soon.",
    
    "I'm anxious about the exam, but I'm also excited about the challenge.",
    
    "I'm surprised and disappointed by the news.",
]

ja_mixed_emotion_sentences = [
    "彼の成功は素晴らしいけれども、同時に少し嫉妬心がある",
    "仕事が終わったので安心だが、週末のパーティには緊張している",
    "彼女の発表には驚いたが、ちょっと怒りも感じた",
]

def calculate_mixed_emotion_entropy(sentence, embeddings, language='en'):
    
    if language == 'ja':
        concept_vectors = vectorize_jp_concepts(ja_basic_emotion_concepts, embeddings)
        tokens = tokenize_japanese_text_mecab(sentence)
        text_vector = vectorize_jp_text(tokens, embeddings)
    else:
        concept_vectors = vectorize_concepts(en_basic_emotion_concepts, embeddings)
        text_vector = vectorize_text(sentence, embeddings)
    similarities = calculate_similarities(text_vector, concept_vectors)
    entropy = emotion_entropy(list(similarities.values()))
    return similarities, entropy

for sentence in en_mixed_emotion_sentences:
    similarities, entropy = calculate_mixed_emotion_entropy(sentence, en_embeddings)
    print(sentence, similarities, entropy)

for sentence in ja_mixed_emotion_sentences:
    similarities, entropy = calculate_mixed_emotion_entropy(sentence, ja_embeddings, language='ja')
    print(sentence, similarities, entropy)


I'm so happy to see you, but I'm sad that you have to leave soon. {'fear': 0.37510133, 'sadness': 0.41060042, 'anger': 0.3957128, 'surprise': 0.45528015, 'disgust': 0.38073587, 'joy': 0.43462107} 0
I'm anxious about the exam, but I'm also excited about the challenge. {'fear': 0.3561712, 'sadness': 0.38363206, 'anger': 0.37038356, 'surprise': 0.41533506, 'disgust': 0.35978395, 'joy': 0.3962078} 0
I'm surprised and disappointed by the news. {'fear': 0.37021977, 'sadness': 0.3916921, 'anger': 0.38696092, 'surprise': 0.420673, 'disgust': 0.36965284, 'joy': 0.38753802} 0
彼の成功は素晴らしいけれども、同時に少し嫉妬心がある {'anger': 0.65449595, 'disgust': 0.6265899, 'fear': 0.6435342, 'joy': 0.7127478, 'sadness': 0.65797377, 'surprise': 0.7134285} 2.584962500721156
仕事が終わったので安心だが、週末のパーティには緊張している {'anger': 0.5855845, 'disgust': 0.5570482, 'fear': 0.57905364, 'joy': 0.647285, 'sadness': 0.59128726, 'surprise': 0.6525617} 2.584962500721156
彼女の発表には驚いたが、ちょっと怒りも感じた {'anger': 0.61370766, 'disgust': 0.5839083, 'fear': 0.5991

In [12]:
mixed_emotion_sentences = {
    "sentence1": "I'm thrilled to have received the promotion, but I'm also nervous about the new responsibilities that come with it.",
    "sentence2": "I was relieved that the medical test came back negative, but I'm still worried about my persistent symptoms.",
    "sentence3": "I'm excited to start this new chapter in my life, but I'm sad about leaving my friends and family behind.",
    "sentence4": "It's heartbreaking to say goodbye to my colleagues, but I am eager to explore the new job opportunity.",
    "sentence5": "I'm proud of my children for becoming independent, but I'm also feeling a little lonely now that they've moved out.",
    "sentence6": "I feel joyous for my friend's wedding, but at the same time, I'm feeling a bit envious.",
}
en_mixed_emotion_sentences = list(mixed_emotion_sentences.values())

ja_mixed_emotion_sentences= [
    "昇進してとてもうれしいですが、新しい責任にも少し緊張しています。",
    "医療検査が陰性でほっとしていますが、持続的な症状についてはまだ心配しています。",
    "新しい生活の章を開始するのが楽しみですが、友人や家族と離れることは悲しいです。",
    "同僚に別れを告げるのは心が痛むけれど、新たな仕事の機会を探求することには前向きです。",
    "子どもたちが自立してくれて誇らしいですが、彼らが引っ越してからは少し寂しい気もします。",
    "友人の結婚に喜んでいる一方で、少し羨ましい気もします。",
]


for sentence in en_mixed_emotion_sentences:
    similarities, entropy = calculate_mixed_emotion_entropy(sentence, en_embeddings)
    print(sentence, similarities, entropy)

for sentence in ja_mixed_emotion_sentences:
    similarities, entropy = calculate_mixed_emotion_entropy(sentence, ja_embeddings, language='ja')
    print(sentence, similarities, entropy)



I'm thrilled to have received the promotion, but I'm also nervous about the new responsibilities that come with it. {'fear': 0.401712, 'sadness': 0.42628855, 'anger': 0.41740072, 'surprise': 0.47821116, 'disgust': 0.39795023, 'joy': 0.45706573} 0
I was relieved that the medical test came back negative, but I'm still worried about my persistent symptoms. {'fear': 0.43635708, 'sadness': 0.46855152, 'anger': 0.44494754, 'surprise': 0.49110746, 'disgust': 0.43099713, 'joy': 0.45792305} 0
I'm excited to start this new chapter in my life, but I'm sad about leaving my friends and family behind. {'fear': 0.41011935, 'sadness': 0.44670117, 'anger': 0.42168635, 'surprise': 0.48029295, 'disgust': 0.40653467, 'joy': 0.47900423} 0
It's heartbreaking to say goodbye to my colleagues, but I am eager to explore the new job opportunity. {'fear': 0.39935225, 'sadness': 0.42127663, 'anger': 0.41590923, 'surprise': 0.48764572, 'disgust': 0.3840256, 'joy': 0.4741584} 0
I'm proud of my children for becoming 

# caculate tweets entropy

In [15]:
from tqdm import tqdm

entropies = []
angers = []
fears = []
joys = []
sadnesses = []
disgusts = []
surprises = []

for t in tqdm(usdf['text']):
    text_vector = vectorize_text(t, en_embeddings)

    similarities = calculate_similarities(text_vector, concept_vectors)
    # similarities, entroppy = calculate_mixed_emotion_entropy(t, en_embeddings)
    # entropies.append(entropy)
    angers.append(similarities['anger'])
    fears.append(similarities['fear'])
    joys.append(similarities['joy'])
    sadnesses.append(similarities['sadness'])
    disgusts.append(similarities['disgust'])
    surprises.append(similarities['surprise'])

# usdf['entropy'] = entropies
usdf['anger'] = angers
usdf['fear'] = fears
usdf['joy'] = joys
usdf['sadness'] = sadnesses
usdf['disgust'] = disgusts
usdf['surprise'] = surprises

  2%|▏         | 25630/1471856 [1:21:32<70:01:16,  5.74it/s] 

In [68]:
entropy

0

In [ ]:
angers = []
fears = []
joys = []
sadnesses = []
disgusts = []
surprises = []

for t in tqdm(jadf['text']):
    tokens = tokenize_japanese_text_mecab(t)
    text_vector = vectorize_jp_text(tokens, ja_embeddings)
    similarities = calculate_similarities(text_vector, concept_vectors)
    # similarities, entroppy = calculate_mixed_emotion_entropy(t, en_embeddings)
    # entropies.append(entropy)
    angers.append(similarities['anger'])
    fears.append(similarities['fear'])
    joys.append(similarities['joy'])
    sadnesses.append(similarities['sadness'])
    disgusts.append(similarities['disgust'])
    surprises.append(similarities['surprise'])

jadf['anger'] = angers
jadf['fear'] = fears
jadf['joy'] = joys
jadf['sadness'] = sadnesses
jadf['disgust'] = disgusts
jadf['surprise'] = surprises


In [ ]:
# save the data
usdf.to_csv(os.path.join(preprocessed_folder_path,"us_entropy.csv") index=False)
jpdf.to_csv(os.path.join(preprocessed_folder_path,"jp_entropy.csv") , index=False)

In [ ]:
# compare the entropy of the two languages
print(usdf['entropy'].mean())
print(jpdf['entropy'].mean())

# test which is higher
from scipy.stats import ttest_ind
ttest_ind(usdf['entropy'], jpdf['entropy'])